In [11]:
!pip install pandas -qq

In [7]:
import gdown
file_id = '1MAXmmYzW1WUUDyNZBCiZNdP2njZFHUL0'
file_path = './dataset.zip'

gdown.download(f'https://drive.google.com/uc?id={file_id}', file_path, quiet=False)
!unzip ./dataset.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1MAXmmYzW1WUUDyNZBCiZNdP2njZFHUL0
From (redirected): https://drive.google.com/uc?id=1MAXmmYzW1WUUDyNZBCiZNdP2njZFHUL0&confirm=t&uuid=e2509643-4440-40f6-b0ce-dee3b238f200
To: /home/ubuntu/hive_retrieval_engine/dataset.zip
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 56.4M/56.4M [00:00<00:00, 65.8MB/s]

Archive:  ./dataset.zip


  inflating: categories.npy          
  inflating: training_data.npy       
  inflating: valid_periods.npy       


In [1]:
import numpy as np
categories = np.load('./categories.npy')
valid_periods = np.load('./valid_periods.npy')
training_data = np.load('./training_data.npy')

In [2]:
mask = categories == 'B'

In [3]:
valid_periods = valid_periods[mask]
training_data = training_data[mask]

In [4]:
import pandas as pd
training_data_df = pd.DataFrame(training_data)
training_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10987 entries, 0 to 10986
Columns: 2776 entries, 0 to 2775
dtypes: float64(2776)
memory usage: 232.7 MB


In [5]:
from sklearn.model_selection import train_test_split

# Split the indices of the time series into training and validation sets
train_indices, val_indices = train_test_split(np.arange(len(training_data_df)), test_size=0.2, random_state=42)

# Use these indices to create training and validation sets
train_set = training_data_df.iloc[train_indices]
valid_periods_train = valid_periods[train_indices]

val_set = training_data_df.iloc[val_indices]
valid_periods_val = valid_periods[val_indices]

train_set.shape, valid_periods_train.shape, val_set.shape, valid_periods_val.shape

((8789, 2776), (8789, 2), (2198, 2776), (2198, 2))

In [6]:
def calculate_padding_length(sq, w, t, s):
    cycle = w + t
    if sq <= cycle: return cycle - sq
    else:
        m = (sq - cycle) % s
        if m == 0: return m
        else: return s - m

In [7]:
def create_sequences(df, valid_periods, window, telescope, stride=1):

    input_sequences = []
    input_categories = []
    output_sequences = []


    for i in range(len(valid_periods)):
        start, end = valid_periods[i]

        sequence = df.iloc[i, start:end].values

        padding_length = calculate_padding_length(len(sequence), window, telescope, stride)
        sequence = np.pad(sequence, (padding_length, 0), mode='constant', constant_values=0)

        for j in range(0, len(sequence) - window - telescope + 1, stride):
            input_seq = sequence[j:(j + window)]
            output_seq = sequence[(j + window):(j + window + telescope)]

            input_sequences.append(input_seq)
            output_sequences.append(output_seq)

    return np.array(input_sequences), np.array(output_sequences)

In [8]:
window = 200
telescope = 9
stride = 10

In [9]:
X_train, y_train = create_sequences(train_set, valid_periods_train, window, telescope, stride)
X_val, y_val = create_sequences(val_set, valid_periods_val, window, telescope, stride)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((33912, 200), (33912, 9), (8597, 200), (8597, 9))

In [10]:
input_shape = (window, 1)
category_shape = (6, )
output_shape = (telescope, 1)
batch_size = 64
epochs = 200

In [11]:
from Transformer import Transformer
model = Transformer(window, telescope, 1)

2023-12-15 21:18:52.595041: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-15 21:18:52.644586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 21:18:53.403485: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
model.train(X_train, y_train, X_val, y_val)

2023-12-15 21:18:54.251512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 21:18:54.291321: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-15 21:18:54.293454: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 200, 1)]             0         []                            
                                                                                                  
 layer_normalization (Layer  (None, 200, 1)               2         ['input_1[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 multi_head_attention (Mult  (None, 200, 1)               7169      ['layer_normalization[0][0]', 
 iHeadAttention)                                                     'layer_normalization[0][0]'] 
                                                                                              

2023-12-15 21:19:07.616429: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-15 21:19:08.425044: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x562c5dcd2bc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-15 21:19:08.425075: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2023-12-15 21:19:08.430580: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-15 21:19:08.556185: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


530/530 [==============================] - ETA: 0s - loss: 0.0323 - rmse: 0.1667 - mae: 0.1245 - smape: 16.4589 - coeff_determination: 0.5552
Epoch 1: val_loss improved from inf to 0.01247, saving model to parameters/Transformer.best15122023_21:18:56.hdf5


/home/ubuntu/hive_retrieval_engine/test_venv/lib/python3.11/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


530/530 [==============================] - 146s 231ms/step - loss: 0.0323 - rmse: 0.1667 - mae: 0.1245 - smape: 16.4589 - coeff_determination: 0.5552 - val_loss: 0.0125 - val_rmse: 0.1091 - val_mae: 0.0725 - val_smape: 10.9043 - val_coeff_determination: 0.7962 - lr: 0.0010
Epoch 2/200
530/530 [==============================] - ETA: 0s - loss: 0.0158 - rmse: 0.1248 - mae: 0.0894 - smape: 12.2076 - coeff_determination: 0.7811
Epoch 2: val_loss improved from 0.01247 to 0.01136, saving model to parameters/Transformer.best15122023_21:18:56.hdf5
530/530 [==============================] - 122s 231ms/step - loss: 0.0158 - rmse: 0.1248 - mae: 0.0894 - smape: 12.2076 - coeff_determination: 0.7811 - val_loss: 0.0114 - val_rmse: 0.1039 - val_mae: 0.0693 - val_smape: 10.3557 - val_coeff_determination: 0.8141 - lr: 0.0010
Epoch 3/200
530/530 [==============================] - ETA: 0s - loss: 0.0138 - rmse: 0.1166 - mae: 0.0818 - smape: 11.4459 - coeff_determination: 0.8087
Epoch 3: val_loss improved